In [6]:
from vega import Vega
import altair as alt
import numpy as np

import pandas as pd
institutions_df = pd.read_csv("institution-totals.csv")

nat_institutions_institutions_df = institutions_df.query('sector == "national"')

# National Museum Institutions

In [7]:
row_count = 0
row_graphs = []
grid_graphs = []

for i, institution in nat_institutions_institutions_df.sort_values(by=["institution"]).iterrows():
    institution_name = institution["institution"]
    title = alt.TitleParams(institution_name, anchor='middle')
    row_df = pd.DataFrame(
       {"institution": institution_name,
        "values": [
             institution["artefacts"],
            institution["objects"], institution["library_items"], institution["archive_items"],
            institution["object_records"], institution["library_records"], institution["archive_records"]],
       "calculated": [
            min(institution["object_records"] + institution["library_records"] + institution["archive_records"], institution["artefacts"]),
            institution["artefacts"],
            min(institution["object_records"], institution["objects"]), institution["objects"],
            min(institution["library_records"], institution["library_items"]), institution["library_items"],
         institution["archive_items"] ],
            "labels": ["Artefacts", 
                                  "Museum Objects", 
                                  "Library Items",
                                  "Archive Items/Collections", 
                                  "Museum Records", 
                                  "Library Records", "Archive Records"],
            "labels_calculated": [ "Artefacts Catalogued", "Artefacts", "Objects Catalogued", "Objects",
                                  "Library Items Catalogued", "Library Items",
                                   "Archive Items"]
            })

    # Ring gives summary of cataloguing progress
    #ring = alt.Chart(row_df).mark_arc(innerRadius=90, outerRadius=100).encode(
    #  color="labels_calculated:N",
    #  theta="calculated",
    #)

    if institution["artefacts"] > 0:
     catalogued_records =  institution["object_records"] + institution["library_records"] + institution["archive_records"]
     if catalogued_records > 0:
         catalogued_proportion = min(catalogued_records / institution["artefacts"], 1) * 2.09
         ring1_complete = alt.Chart().mark_arc(
             tooltip=f'Catalogued Artefacts: {catalogued_records}',
             innerRadius=90, outerRadius=100,theta=0,color="gold", 
                                theta2=catalogued_proportion - 0.02)
         if catalogued_proportion < 2.09:
           ring1_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=catalogued_proportion,color="#9ecae1", 
                                                  theta2=2.07,
                                                  tooltip=f'Total Artefacts: {institution["artefacts"]}')
         else:
           ring1_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=0,color="grey", theta2=0)
     else:
        ring1_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=0,color="#9ecae1", theta2=0)
        ring1_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=0,theta2=2.07,color="#9ecae1",
                                               tooltip=f'Total Artefacts: {institution["artefacts"]}')
    else:
      ring1_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=0,color="grey", theta2=0)
      ring1_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=0,theta2=2.07,color="grey",
                                             tooltip=f'Total Artefacts: {institution["artefacts"]}')


    if institution["objects"] > 0:
     catalogued_records =  institution["object_records"] 
     if catalogued_records > 0:
         catalogued_proportion = min(catalogued_records / institution["objects"], 1) * 2.09
         ring2_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,color="gold",
                                               theta2=2.09 + catalogued_proportion - 0.02, 
                                                            tooltip=f'Catalogued Objects: {catalogued_records}')
         if catalogued_proportion < 2.09:
           ring2_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09+ catalogued_proportion,color="#fd8d3c", 
                                                  theta2=4.16, tooltip=f'Total Objects: {institution["objects"]}')
     else:
        ring2_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,color="#fd8d3c", theta2=2.09)
        ring2_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,theta2=4.16,color="#fd8d3c",
                                               tooltip=f'Total Objects: {institution["objects"]}')
    else:
      ring2_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,color="grey", theta2=2.09)
      ring2_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,theta2=4.16,color="grey", 
                                             tooltip=f'Total Objects: {institution["objects"]}')  
        
#    ring2 = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=2.09,theta2=4.16,color="gray")

    if institution["library_items"] > 0:
     catalogued_records = institution["library_records"] 
     if catalogued_records > 0:
         catalogued_proportion = min(catalogued_records / institution["library_items"], 1) * 2.09
         ring3_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,
                                               theta=4.18,color="gold", 
                                               tooltip=f'Catalogued Library Items: {catalogued_records}',
                                               theta2=4.18 + catalogued_proportion - 0.02)
         if catalogued_proportion < 2.09:
           ring3_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18+catalogued_proportion,color="#c6dbef", 
                                                  theta2=6.27, tooltip=f'Total Library Items: {institution["library_items"]}')
     else:
        ring3_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18,color="#c6dbef", theta2=4.18)
        ring3_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18,theta2=6.27,
                                               color="#c6dbef", tooltip=f'Total Library Items: {institution["library_items"]}')
    else:
      ring3_complete = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18,color="grey", theta2=4.18)
      ring3_remaining = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18,theta2=6.27,color="grey",
                                             tooltip=f'Total Library Items: {institution["library_items"]}') 
        
#    ring3 = alt.Chart().mark_arc(innerRadius=90, outerRadius=100,theta=4.18,theta2=6.27,color="orange")

    # Petal graph ? gives summary of size of instution collections
    base1 = alt.Chart(row_df, width=160, title=title).encode(
      alt.Theta("values:Q").stack(True),
      alt.Radius("values").scale(type="sqrt", zero=True, rangeMin=20),
      tooltip='labels:N',
      color=alt.Color("labels:N", sort="ascending").legend(orient="top", titleOrient="left").scale(scheme="category20c"),
    ).properties(
      width=200,
      height=200
    )

#    text_complete = alt.Chart().mark_text().encode(
#        text=f"{institution['artefacts']}:Q")
#        text=alt.Text(institution["artefacts"]), color="gold").properties(width=100)
    
    petal_marks = base1.mark_arc(innerRadius=45, outerRadius=60, cornerRadius=5, 
                                 stroke="#fff")
    
#    petal_text = base1.mark_text(radiusOffset=58).encode(text="values:Q")
#    alt.FieldGTPredicate("values", 0), alt.Text('values:Q'), alt.value(''))

    petal_text = base1.mark_text(radiusOffset=58).encode(
        text=alt.condition(alt.FieldGTPredicate("values", 0), alt.Text('values:Q'), alt.value('')))

    # Bar at bottom indicates proportion of national collection this institution has
    bar = alt.Chart(nat_institutions_institutions_df, height=20, width=200).mark_bar().encode(
      alt.X("artefacts").title("Proportion of National Collection"),
      tooltip='artefacts',
      color=alt.condition(
        alt.FieldOneOfPredicate('institution', [institution_name]),
        alt.value('gold'),     
        alt.value('white')     
      )
    )
    
    institution_graph = alt.vconcat((petal_marks + petal_text + ring1_complete + 
                                     ring1_remaining + ring2_complete + ring2_remaining +
                                     ring3_remaining + ring3_complete), bar).resolve_scale(theta='independent')
    if row_count < 3:
        row_graphs.append(institution_graph)
        row_count += 1
    else:   
        row_graphs.append(institution_graph)
        grid_graphs.append(
            alt.hconcat(*row_graphs, spacing=40).resolve_scale(theta='shared'))
        row_count = 0
        row_graphs = []

if row_count > 0:
    grid_graphs.append(
            alt.hconcat(*row_graphs, spacing=40).resolve_scale(theta='shared'))

alt.vconcat(*grid_graphs).configure(numberFormat='.2s')
#alt.hconcat(*row_graphs, spacing=40).configure(numberFormat='.2s').resolve_scale(theta='shared')

alt.VConcatChart(...)